## 准备数据

In [2]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import numpy as np
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    x = x.reshape(x.shape[0], 28, 28,1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28,1)
    
#     train_label = np.zeros(shape=[y.shape[0], 10])
#     test_label = np.zeros(shape=[y_test.shape[0], 10])
#     train_label[np.arange(y.shape[0]), np.array(y)] = 1.
#     test_label[np.arange(y_test.shape[0]), np.array(y_test)] = 1.
    
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(prepare_mnist_features_and_labels)
    ds = ds.take(20000).shuffle(20000).batch(100)
    
    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_ds = test_ds.map(prepare_mnist_features_and_labels)
    test_ds = test_ds.take(20000).shuffle(20000).batch(20000)
    return ds, test_ds

def prepare_mnist_features_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y

c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\ten

# 有点小问题

#### 建立模型

In [39]:
model = keras.Sequential([
    Conv2D(32, (5, 5), activation='relu', padding='same'),
    MaxPooling2D(pool_size=2, strides=2),
    Conv2D(64, (5, 5), activation='relu', padding='same'),
    MaxPooling2D(pool_size=2, strides=2),
    Flatten(), #N*7*7*64 =>N*3136
    layers.Dense(128, activation='relu'), #N*128
    layers.Dense(10,activation='tanh')]) #N*10
optimizer = optimizers.Adam()

#### 编译， fit以及evaluate

In [40]:
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
train_ds, test_ds = mnist_dataset()

In [41]:
model.fit(train_ds, epochs=5)
model.evaluate(test_ds)

Epoch 1/5
200/200 [==============================] - 2s 12ms/step - loss: 4.1019 - accuracy: 0.1547
Epoch 2/5
200/200 [==============================] - 2s 9ms/step - loss: 2.3026 - accuracy: 0.0894
Epoch 3/5
200/200 [==============================] - 2s 9ms/step - loss: 2.3026 - accuracy: 0.0894
Epoch 4/5
200/200 [==============================] - 2s 9ms/step - loss: 2.3026 - accuracy: 0.0894
Epoch 5/5
1/1 [==============================] - 0s 370ms/step - loss: 2.3026 - accuracy: 0.0892


[2.3025856018066406, 0.0892]

# 成功收敛的结果

## 建立模型

In [3]:
model = keras.Sequential([
    Conv2D(32, (5, 5), activation='relu', padding='same'),
    MaxPooling2D(pool_size=2, strides=2),
    Conv2D(64, (5, 5), activation='relu', padding='same'),
    MaxPooling2D(pool_size=2, strides=2),
    Flatten(), #N*7*7*64 =>N*3136
    layers.Dense(128, activation='relu'), #N*128
    layers.Dense(10,activation='softmax')]) #N*10
optimizer = optimizers.Adam()

## 编译， fit以及evaluate

In [4]:
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',  # sparse_categorical_crossentropy 可以直接传 1，2，3，4 而不需要one-hot 区别于
              metrics=['accuracy'])                    # categorical_crossentropy
train_ds, test_ds = mnist_dataset()

In [5]:
model.fit(train_ds, epochs=5)  # 偶尔会出现奇怪的错误 ，说 输入的数据维度的channel有问题，然而重新运行就好了，不知道为什么
model.evaluate(test_ds)

Epoch 1/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
200/200 [==============================] - 3s 17ms/step - loss: 0.3045 - accuracy: 0.7998
Epoch 2/5
200/200 [==============================] - 2s 9ms/step - loss: 0.0736 - accuracy: 0.9730
Epoch 3/5
200/200 [==============================] - 2s 9ms/step - loss: 0.0456 - accuracy: 0.9830
Epoch 4/5
200/200 [==============================] - 2s 9ms/step - loss: 0.0325 - accuracy: 0.9881
Epoch 5/5
1/1 [==============================] - 1s 890ms/step - loss: 0.0396 - accuracy: 0.9868


[0.03957824781537056, 0.9868]

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  401536    
_________________________________________________________________
dense_1 (Dense)              multiple                  1